## Remapping training data to the cubed sphere

The novel addition in DLWP-CS is the ability to train convolutional neural networks on data mapped to the cubed sphere. The re-mapping is performed offline from the model training/inference. 

#### Required packages

We use the TempestRemap library for cubed sphere remapping which is available as a pre-compiled conda package. Let's start by installing it.

In [ ]:
#%conda install -c conda-forge tempest-remap

Let's use the DLWP CubeSphereRemap class on the data we processed earlier.

In [1]:
#import os
#os.chdir(os.pardir)
#from DLWP.remap import CubeSphereRemap

#data_directory = '/home/disk/wave2/jweyn/Data/ERA5'
#processed_file = '%s/tutorial_z500_t2m.nc' % data_directory
#remapped_file = '%s/tutorial_z500_t2m_CS.nc' % data_directory

#csr = CubeSphereRemap()

import os
#print(os.environ['TEMPESTREMAP'])
TEMPESTREMAP = os.environ['TEMPESTREMAP']
#NOTEBOOK_PATH = "/scratch/fp0/mah900/DLWP-CS" 
#os.chdir (f'{NOTEBOOK_PATH}')
#os.chdir ('/tmp')
os.chdir (f"/scratch/vp91/{os.environ['USER']}/NCI-DLWP-CS")
from DLWP.remap import CubeSphereRemap

#data_directory = '/g/data/wb00/admin/testing/DLWP/Data/NCI_tutorial'
data_directory = f"/scratch/vp91/{os.environ['USER']}/NCI-DLWP-CS/Data/NCI_tutorial"

processed_file = f'{data_directory}/NCI_tutorial_z500_t2m.nc' 
remapped_file  = f'{data_directory}/NCI_tutorial_z500_t2m_CS.nc' 

# csr = CubeSphereRemap(path_to_remapper='/scratch/fp0/mah900/env/dlwp/bin/')
#csr = CubeSphereRemap(path_to_remapper='/scratch/fp0/mah900/tmp/tempest_remap/dlwp/bin/')
#csr = CubeSphereRemap()
#csr = CubeSphereRemap('/g/data/pp66/data/nci-dlwp-cs/dlwp/bin')
csr = CubeSphereRemap(TEMPESTREMAP)

Generate the offline maps. Since we used 2 degree data, we have 91 latitude points and 180 longitude points. We are mapping to a cubed sphere with 48 points on the side of each cube face. Since data from CDS comes with monotonically decreasing latitudes, we specify the `inverse_lat` option. New versions of TempestRemap have added the capability to read the coordinates from a netCDF file with any latitude/longitude coordinate names. You'll need to install it from source to use the `generate_offline_maps_from_file` method.

In [2]:
#csr.generate_offline_maps(lat=91, lon=180, res=48, inverse_lat=True)
csr.generate_offline_maps(lat=32, lon=64, res=48, inverse_lat=True)

CubeSphereRemap: generating offline forward map...
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/GenerateRLLMesh --lat 32 --lon 64 --file outLL.g --lat_begin 90 --lat_end -90 --out_format Netcdf4
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/GenerateCSMesh --res 48 --file outCS.g --out_format Netcdf4
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/GenerateOverlapMesh --a outLL.g --b outCS.g --out ov_LL_CS.g --out_format Netcdf4
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/GenerateOfflineMap --in_mesh outLL.g --out_mesh outCS.g --ov_mesh ov_LL_CS.g --in_np 1 --in_type FV --out_type FV --out_map map_LL32x64_CS48.nc --out_format Netcdf4
CubeSphereRemap: generating offline inverse map...
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/GenerateOverlapMesh --a outCS.g --b outLL.g --out ov_CS_LL.g --out_format Netcdf4
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/GenerateOfflineMap --in_mesh outCS.g --out_mesh outLL.g --ov_mesh ov_CS_LL.g --in_np 1 --in_type FV --out_type FV --out_map map_CS48_LL32x64

Apply the forward map, saving to a temporary file. We specify to operate on the variable `predictors`, which is the only variable in the processed data. TempestRemap is very finicky about metadata in netCDF files, sometimes failing with segmentation faults for no apparent reason. I've found that the most common crash is because it does not like the string coordinate values in the `'varlev'` coordinate. If you used the command in the previous tutorial to produce an extra "nocoord" version of this file, you might *have to* use it here.

In [3]:
csr.remap(processed_file + '.nocoord', '%s/temp.nc' % data_directory, '--var', 'predictors')

CubeSphereRemap: applying forward map...
/g/data/dk92/apps/nci-dlwp-cs/2024.03.29/bin/ApplyOfflineMap --in_data /scratch/vp91/mah900/NCI-DLWP-CS/Data/NCI_tutorial/NCI_tutorial_z500_t2m.nc.nocoord --out_data /scratch/vp91/mah900/NCI-DLWP-CS/Data/NCI_tutorial/temp.nc --map map_LL32x64_CS48.nc --var predictors
CubeSphereRemap: successfully remapped data into /scratch/vp91/mah900/NCI-DLWP-CS/Data/NCI_tutorial/temp.nc


By default, TempestRemap has a 1-dimensional spatial coordinate. We convert the file to 3-dimensional faces (face, height, width). A few other points here:  
- Even if TempestRemap does not crash, it will probably delete the string coordinates, and sometimes the sample time coordinate as well, so it's a good idea to use this feature.  
- We also take advantage of the `chunking` parameter to save data with ideal chunking when using the file for training and evaluating models.

In [4]:
csr.convert_to_faces('%s/temp.nc' % data_directory, 
                     remapped_file,
                     coord_file=processed_file,
                     chunking={'sample': 1, 'varlev': 1})

CubeSphereRemap.convert_to_faces: loading data to memory...
CubeSphereRemap.convert_to_faces: assigning new coordinates to dataset
CubeSphereRemap.convert_to_faces: exporting data to file /scratch/vp91/mah900/NCI-DLWP-CS/Data/NCI_tutorial/NCI_tutorial_z500_t2m_CS.nc...


/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.events-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


CubeSphereRemap.convert_to_faces: successfully exported reformatted data


<xarray.Dataset>
Dimensions:     (face: 6, height: 48, width: 48, sample: 17528, varlev: 2)
Coordinates:
  * face        (face) int64 0 1 2 3 4 5
  * height      (height) int64 0 1 2 3 4 5 6 7 8 ... 39 40 41 42 43 44 45 46 47
  * width       (width) int64 0 1 2 3 4 5 6 7 8 9 ... 39 40 41 42 43 44 45 46 47
  * sample      (sample) datetime64[ns] 2013-01-01 ... 2018-12-31T21:00:00
  * varlev      (varlev) object 'z/500' 't2m/0'
Data variables:
    lon         (face, height, width) float64 dask.array<chunksize=(6, 48, 48), meta=np.ndarray>
    lat         (face, height, width) float64 dask.array<chunksize=(6, 48, 48), meta=np.ndarray>
    predictors  (sample, varlev, face, height, width) float32 dask.array<chunksize=(1, 1, 6, 48, 48), meta=np.ndarray>
Attributes:
    description:  Training data for DLWP
    scaling:      True

In [5]:
import os
os.remove('%s/temp.nc' % data_directory)